# CSX46 - Class Session 5 - Components

In this class session we are going to find the number of proteins that are in the giant component of the (undirected) protein-protein interaction network, using igraph.

In [1]:
from igraph import Graph
from igraph import summary
import pandas
import numpy

Step 1:  load in the Pathway Commons SIF file into a data frame `sif_data`, using the `pandas.read_csv` function, and name the columns `species1`, `interaction_type`, and `species2`.

In [2]:
sif_data = pandas.read_csv("shared/pathway_commons.sif",
                           sep="\t", names=["species1","interaction_type","species2"])

Step 2:  restrict the interactions to protein-protein undirected ("in-complex-with", "interacts-with"), by using the `isin` function and then using `[` to index rows into the data frame.  Call the returned ata frame `interac_ppi`.

In [3]:
interaction_types_ppi = set(["interacts-with",
                             "in-complex-with"])
interac_ppi = sif_data[sif_data.interaction_type.isin(interaction_types_ppi)].copy()

Step 3: restrict the data frame to only the unique interaction pairs of proteins (ignoring the interaction type), and call that data frame `interac_ppi_unique`. Make an igraph `Graph` object from `interac_ppi_unique` using `Graph.TupleList`, `values`, and `tolist`. Call `summary` on the `Graph` object. Refer to the notebooks for the in-class exercises in Class sessions 3 and 6.

In [4]:
boolean_vec = interac_ppi['species1'] > interac_ppi['species2']
interac_ppi.loc[boolean_vec, ['species1', 'species2']] = interac_ppi.loc[boolean_vec, ['species2', 'species1']].values
        
interac_ppi_unique = interac_ppi[["species1","species2"]].drop_duplicates()        


ppi_igraph = Graph.TupleList(interac_ppi_unique.values.tolist(), directed=False)
summary(ppi_igraph)

IGRAPH UN-- 17020 491784 -- 
+ attr: name (v)


Step 4: Map the components of the network using the `igraph.Graph.clusters` method. That method returns a `igraph.clustering.VertexClustering` object. Assign it to object `ppi_components`.

In [5]:
ppi_components = ppi_igraph.clusters()

Call the `sizes` method on that `VertexClustering` object, to get a list of sizes of the components.  What is the giant component size?

In [6]:
# call the `sizes` method on the `ppi_components` object, and assign the
# resulting list object to have the name `ppi_component_sizes`.
ppi_component_sizes = ppi_components.sizes()

# make a `numpy.array` initialized by `ppi_component_sizes`, and find its 
# maximum value using the `max` method on the `numpy.array` class
numpy.array(ppi_component_sizes).max()

16985

Let's print out all the component sizes, in reverse-sorted order

In [7]:
sorted(ppi_component_sizes, reverse=True)

[16985, 10, 4, 4, 3, 3, 3, 2, 2, 2, 2]

Let's get the vertex indices of the vertices that are in the giant component using list comprehension, `enumerate`, and the `membership` field on the `ppi_components` object.

In [8]:
inds_giant_component =  [index for index, item in enumerate(ppi_components.membership) if item==numpy.argmax(ppi_component_sizes)]


Let's compute the vertex degrees, for all the vertices in the giant component

In [9]:
ppi_degrees_gc = ppi_igraph.degree(inds_giant_component)

What is the highest degree vertex in the giant component, and what is its degree?

In [10]:
ind_max = numpy.argmax(ppi_degrees_gc)
print("Vertex number {} has maximum degree; it is degree: {}".format(
      ind_max,
      ppi_degrees_gc[ind_max]))


Vertex number 32 has maximum degree; it is degree: 3600


Let's compute the shortest paths distance between highest-degree vertex of the giant component, and all the other vertices in the giant component, using the `shortest_paths` method on the `ppi_igraph` method, specifying both `source` and `target` as lists of vertex IDs.

In [11]:
apsp_dists = ppi_igraph.shortest_paths(source=inds_giant_component[ind_max],
                                       target=inds_giant_component)

What is the average distance between the highest-degree vertex of the giant component and all the other vertices of the giant component? Use `numpy.mean`.

In [12]:
print("%0.2f" % numpy.mean(apsp_dists))

1.86


Code-spellunking question:  go to the GitHub repo for igraph (https://github.com/igraph), and find the code components.c.  For the weakly connected components, is it doing a BFS or DFS?